In [57]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

from sklearn import preprocessing
from sklearn import metrics

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="ticks")

import warnings
warnings.filterwarnings('ignore')

In [65]:
# Import data 
df_sns = pd.read_csv("../OUTPUT/benchmark_sns", sep = "\t")
df_sg = pd.read_csv("..//OUTPUT/benchmark_sg", sep = "\t")

In [66]:
# Init dataset
X = df_sns[['Subject', 'Donor', 'family', 'order', 'class', 'phyla', 
            'Time', 'baseline_abundance', 
            'donor_abundance', 'mean_relab_hmp2012']]
y = df_sns[['Status']]

# Re-format string features
le = preprocessing.LabelEncoder()

for i in range(6):
    X.iloc[:,i] = le.fit_transform(X.iloc[:,i])

# Split train and test data frame
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [67]:
# make griid_search 
n_estimators = [int(x) for x in np.linspace(start = 1, stop = 500, num = 500)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 20, num = 20)]
max_depth.append(None)
min_samples_split = [2, 5, 10, 11]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
criterion = ['entropy']

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion' : criterion}

# Run greed search Random Forest
rf = RandomForestClassifier()
rf_random_sns = RandomizedSearchCV(estimator = rf,
                                   param_distributions = random_grid, 
                                   n_iter = 1000, cv = 10, verbose=2, random_state=42, 
                                   n_jobs = 40)
rf_random_sns.fit(X_train, y_train)

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=40)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done  82 tasks      | elapsed:    4.7s
[Parallel(n_jobs=40)]: Done 285 tasks      | elapsed:   11.8s
[Parallel(n_jobs=40)]: Done 568 tasks      | elapsed:   21.5s
[Parallel(n_jobs=40)]: Done 933 tasks      | elapsed:   31.7s
[Parallel(n_jobs=40)]: Done 1378 tasks      | elapsed:   46.7s
[Parallel(n_jobs=40)]: Done 1905 tasks      | elapsed:  1.1min
[Parallel(n_jobs=40)]: Done 2512 tasks      | elapsed:  1.4min
[Parallel(n_jobs=40)]: Done 3201 tasks      | elapsed:  1.7min
[Parallel(n_jobs=40)]: Done 3970 tasks      | elapsed:  2.1min
[Parallel(n_jobs=40)]: Done 4821 tasks      | elapsed:  2.6min
[Parallel(n_jobs=40)]: Done 5752 tasks      | elapsed:  3.1min
[Parallel(n_jobs=40)]: Done 6765 tasks      | elapsed:  3.7min
[Parallel(n_jobs=40)]: Done 7858 tasks      | elapsed:  4.3min
[Parallel(n_jobs=40)]: Done 9033 tasks      | elapsed:  4.9min
[Parallel(n_jobs=40)]: Done 10000 out of 1000

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
              

In [68]:
group = ["Taxonomy", "Taxonomy", "Taxonomy", "Taxonomy", 
         "Metadata", "Metadata", "Metadata", 
         "Abundance", "Abundance", "Abundance"]
data_imp = {'Feature':  X.columns,
        'Importance': rf_random_sns.best_estimator_.feature_importances_,
         'Group' : group
        }
feature_importance_sns = pd.DataFrame(data_imp, columns = ['Feature','Importance', 'Group'])

In [69]:
# Calculate metrics
f1_score_sns = metrics.f1_score(rf_random_sns.best_estimator_.predict(X_test), y_test)
roc_auc_score_sns = metrics.roc_auc_score(rf_random_sns.best_estimator_.predict(X_test), y_test)
accuracy_score_sns = metrics.accuracy_score(rf_random_sns.best_estimator_.predict(X_test), y_test)
precision_score_sns = metrics.precision_score(rf_random_sns.best_estimator_.predict(X_test), y_test)
recall_score_sns = metrics.recall_score(rf_random_sns.best_estimator_.predict(X_test), y_test)

In [76]:
# Init dataset
X = df_sg[['Subject', 'Donor', 'family', 'order', 'class', 'phyla', 
            'Time', 'baseline_abundance', 
            'donor_abundance', 'mean_relab_hmp2012']]
y = df_sg[['Status']]

# Re-format string features
le = preprocessing.LabelEncoder()

for i in range(6):
    X.iloc[:,i] = le.fit_transform(X.iloc[:,i])

# Split train and test data frame
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# make griid_search 
n_estimators = [int(x) for x in np.linspace(start = 1, stop = 500, num = 500)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 20, num = 20)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
criterion = ['entropy']

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}

# Run greed search Random Forest
rf = RandomForestClassifier()
rf_random_sg = RandomizedSearchCV(estimator = rf, 
                                  param_distributions = random_grid, n_iter = 1000, 
                                  cv = 10, verbose=2, random_state=42, 
                                  n_jobs = 40)
rf_random_sg.fit(X_train, y_train)

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=40)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done  82 tasks      | elapsed:    6.3s
[Parallel(n_jobs=40)]: Done 285 tasks      | elapsed:   14.0s
[Parallel(n_jobs=40)]: Done 568 tasks      | elapsed:   23.1s
[Parallel(n_jobs=40)]: Done 933 tasks      | elapsed:   32.2s
[Parallel(n_jobs=40)]: Done 1378 tasks      | elapsed:   50.0s
[Parallel(n_jobs=40)]: Done 1905 tasks      | elapsed:  1.1min
[Parallel(n_jobs=40)]: Done 2512 tasks      | elapsed:  1.4min
[Parallel(n_jobs=40)]: Done 3201 tasks      | elapsed:  1.8min
[Parallel(n_jobs=40)]: Done 3970 tasks      | elapsed:  2.3min
[Parallel(n_jobs=40)]: Done 4821 tasks      | elapsed:  2.8min
[Parallel(n_jobs=40)]: Done 5752 tasks      | elapsed:  3.3min
[Parallel(n_jobs=40)]: Done 6765 tasks      | elapsed:  3.9min
[Parallel(n_jobs=40)]: Done 7858 tasks      | elapsed:  4.5min
[Parallel(n_jobs=40)]: Done 9033 tasks      | elapsed:  5.2min
[Parallel(n_jobs=40)]: Done 10000 out of 1000

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
              

In [79]:
# Make Feature Importance DataFrame
group = ["Taxonomy", "Taxonomy", "Taxonomy", "Taxonomy", 
         "Metadata", "Metadata", "Metadata", 
         "Abundance", "Abundance", "Abundance"]
data_imp = {'Feature':  X.columns,
        'Importance': rf_random_sg.best_estimator_.feature_importances_,
         'Group' : group
        }

feature_importance_sg = pd.DataFrame(data_imp, columns = ['Feature','Importance', 'Group'])

In [80]:
# Calculate metrics
f1_score_sg = metrics.f1_score(rf_random_sg.best_estimator_.predict(X_test), y_test)
roc_auc_score_sg = metrics.roc_auc_score(rf_random_sg.best_estimator_.predict(X_test), y_test)
accuracy_score_sg = metrics.accuracy_score(rf_random_sg.best_estimator_.predict(X_test), y_test)
precision_score_sg = metrics.precision_score(rf_random_sg.best_estimator_.predict(X_test), y_test)
recall_score_sg = metrics.recall_score(rf_random_sg.best_estimator_.predict(X_test), y_test)

In [81]:
print(rf_random_sns.best_estimator_)
print(rf_random_sg.best_estimator_)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=239,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                      

In [86]:
data_auc = {'Dataset':  ['settle', 'stay'],
        'AUC': [roc_auc_score_sns, roc_auc_score_sg],
        'precision' : [precision_score_sns, precision_score_sg],
        'accuracy' : [accuracy_score_sns, accuracy_score_sg],
         'recall' : [recall_score_sns, recall_score_sg],
            'f1': [f1_score_sns, f1_score_sg]
        }
data_auc = pd.DataFrame(data_auc, columns = ['Dataset', 'AUC', 'recall', 'precision', 'accuracy', 'f1'])

In [87]:
feature_importance_sns['Sort'] = pd.Series('Settle', index=feature_importance_sns.index)
feature_importance_sg['Sort'] = pd.Series('Stay', index=feature_importance_sg.index)

feature_importance_sns = feature_importance_sns.sort_values('Importance', ascending = False)
feature_importance_sg = feature_importance_sg.sort_values('Importance', ascending = False)

feature_importance = feature_importance_sns.append(feature_importance_sg)

In [88]:
data_auc.to_csv ('../OUTPUT/benchmark_rf_metrics_scores', index = False, header=True)
feature_importance.to_csv ('../OUTPUT/benchmark_rf_feature_importance', index = False, header=True)